In [1]:
# ============================================================
# Notebook setup
# ============================================================

%load_ext autoreload
%autoreload 2

# Control figure size
figsize=(14, 4)

from util import util
import os
import numpy as np
import pandas as pd
data_folder = os.path.join('..', 'data')
file_name = "Data"

# Creating a context

To predict a match outcome, we need to understand the how the two teams compare. This includes both long term form and short term form. We want to create a Match Context that describes this information. To do so we will create new columns.

## Feature Engineering

An important part of pre-processing is to engineer good features that can provide insight into the data and support good predictions. If we want to predict an outcome, we need to set up a dataset that has a defined X and y set of columns, where the X-features has predictive power of the y-column. In our case, the set of features in X will be our Match Context, while the y-column will be the match outcome, represented by 1, 0 or -1, where 1 is a home win, 0 is a draw and -1 is an away win.

### Load data

In [32]:
data = util.load_data(data_folder, file_name)
data = util.clean_data(data)
data.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AST,HF,AF,HC,AC,HY,AY,HR,AR,Season
0,E0,2005-08-13,Aston Villa,Bolton,2.0,2.0,D,2.0,2.0,D,...,6.0,14.0,16.0,7.0,8.0,0.0,2.0,0.0,0.0,0506
1,E0,2005-08-13,Everton,Man United,0.0,2.0,A,0.0,1.0,A,...,5.0,15.0,14.0,8.0,6.0,3.0,1.0,0.0,0.0,0506
2,E0,2005-08-13,Fulham,Birmingham,0.0,0.0,D,0.0,0.0,D,...,4.0,12.0,13.0,6.0,6.0,1.0,2.0,0.0,0.0,0506
3,E0,2005-08-13,Man City,West Brom,0.0,0.0,D,0.0,0.0,D,...,3.0,13.0,11.0,3.0,6.0,2.0,3.0,0.0,0.0,0506
4,E0,2005-08-13,Middlesbrough,Liverpool,0.0,0.0,D,0.0,0.0,D,...,7.0,17.0,11.0,5.0,0.0,2.0,3.0,1.0,0.0,0506


### Add ELO ratings to the dataset

In [33]:
home_factor, draw_factor, away_factor = data['FTR'].value_counts(normalize=True)['H'], data['FTR'].value_counts(normalize=True)['D'], data['FTR'].value_counts(normalize=True)['A']
ELO = util.ELO(data, init_rating=1500, draw_factor=draw_factor, k_factor=32, home_advantage=50)
data = ELO.perform_simulations(data)
data = ELO.get_probabilities(data)

## Add new features

To better describe the strength difference between the two team we create the following features:

- **`Diff_goals_scored`**: The difference in goals scored for the last 5 matches between the two teams
- **`Diff_goals_conceded`**: The difference in goals conceded for the last 5 matches between the two teams
- **`Diff_goal_diff`**: The difference in goal difference for the last 5 matches between the two teams
- **`Diff_points`**: The difference in points for the last 5 matches between the two teams
- **`Diff_change_in_ELO`**: The difference in change in ELO for the last 5 matches between the two teams
- **`Diff_opposition_mean_ELO`**: The difference in the mean ELO of the opposition for the last 5 matches between the two teams
- **`Diff_shots_on_target_attempted`**: The difference in shots on target attempted for the last 5 matches between the two teams
- **`Diff_shots_on_target_allowed`**: The difference in shots on target allowed for the last 5 matches between the two teams
- **`Diff_shots_attempted`**: The difference in shots attempted for the last 5 matches between the two teams
- **`Diff_shots_allowed`**: The difference in shots allowed for the last 5 matches between the two teams
- **`Diff_corners_awarded`**: The difference in corners awarded for the last 5 matches between the two teams
- **`Diff_corners_conceded`**: The difference in corners conceded for the last 5 matches between the two teams
- **`Diff_fouls_committed`**: The difference in fouls committed for the last 5 matches between the two teams
- **`Diff_fouls_suffered`**: The difference in fouls suffered for the last 5 matches between the two teams
- **`Diff_yellow_cards`**: The difference in yellow cards for the last 5 matches between the two teams
- **`Diff_red_cards`**: The difference in red cards for the last 5 matches between the two teams

### Adding features to describe goals

In [34]:
#Add goals scored last five games for both home and away team
data = util.add_form_column(data, 'FTHG', 'FTAG', n=5, operation='Sum', regard_opponent=False, include_current=False)

#Add difference in goals scored last five games between the two teams
data['Diff_goals_scored'] = data['FTHG_Sum_5'] - data['FTAG_Sum_5']



#Add goals conceded last five games for both home and away team
data = util.add_form_column(data, 'FTHG', 'FTAG', n=5, operation='Sum', regard_opponent=True, include_current=False)

#Add difference in goals conceded last five games between the two teams
data['Diff_goals_conceded'] = data['FTHG_Sum_5_opponent'] - data['FTAG_Sum_5_opponent']



#Add goal difference last five games for home and away team
data['Home Goal Difference last 5'] = data['FTHG_Sum_5'] - data['FTHG_Sum_5_opponent']
data['Away Goal Difference last 5'] = data['FTAG_Sum_5'] - data['FTAG_Sum_5_opponent']

#Add difference in goal difference last five games bwteen the two teams
data['Matchrating'] = data['Home Goal Difference last 5'] - data['Away Goal Difference last 5']

### Adding features to describe points

In [35]:
#Add points scored last 5 games for both home and away team
data = util.add_form_column(data, 'Home', 'Away', n=5, operation='Points', regard_opponent=False, include_current=False)

#Add difference in points last 5 games for home and away team
data['Diff_points'] = data['Home_Points_5'] - data['Away_Points_5']

### Adding features to describe ELO from previous games

In [36]:
#Add total change in ELO for the last five games for both home and away team
data = util.add_form_column(data, 'Home ELO', 'Away ELO', n=5, operation='Change', regard_opponent=False, include_current=True)

#Add difference in ELO-change last 5 games between the two teams
data['Diff_change_in_ELO'] = data['Home ELO_Change_5'] - data['Away ELO_Change_5']



#Add the mean ELO of the opponent for the last five games for both home and away team
data = util.add_form_column(data, 'Home ELO', 'Away ELO', n=5, operation='Mean', regard_opponent=True, include_current=False)

#Add difference in mean ELO of the opponent last 5 games between the two teams
data['Diff_opposition_mean_ELO'] = data['Home ELO_Mean_5_opponent'] - data['Away ELO_Mean_5_opponent']

### Adding features to describe other stats

In [37]:
#Add shots on target attempted last five games for both home and away team
data = util.add_form_column(data, 'HST', 'AST', n=5, operation='Sum', regard_opponent=False, include_current=False)

#Add difference in shots on target attempted last five games between the two teams
data['Diff_shots_on_target_attempted'] = data['HST_Sum_5'] - data['AST_Sum_5']


#Add shots on target allowed last five games for both home and away team
data = util.add_form_column(data, 'HST', 'AST', n=5, operation='Sum', regard_opponent=True, include_current=False)

#Add difference in shots on target allowed last five games between the two teams
data['Diff_shots_on_target_allowed'] = data['HST_Sum_5_opponent'] - data['AST_Sum_5_opponent']


#Add shots attempted last five games for both home and away team
data = util.add_form_column(data, 'HS', 'AS', n=5, operation='Sum', regard_opponent=False, include_current=False)

#Add difference in shots attempted last five games between the two teams
data['Diff_shots_attempted'] = data['HS_Sum_5'] - data['AS_Sum_5']


#Add shots allowed last five games for both home and away team
data = util.add_form_column(data, 'HS', 'AS', n=5, operation='Sum', regard_opponent=True, include_current=False)

#Add difference in shots allowed last five games between the two teams
data['Diff_shots_allowed'] = data['HS_Sum_5_opponent'] - data['AS_Sum_5_opponent']


#Add corners awarded last five games for both home and away team
data = util.add_form_column(data, 'HC', 'AC', n=5, operation='Sum', regard_opponent=False, include_current=False)

#Add difference in corners awarded last five games between the two teams
data['Diff_corners_awarded'] = data['HC_Sum_5'] - data['AC_Sum_5']


#Add corners conceded last five games for both home and away team
data = util.add_form_column(data, 'HC', 'AC', n=5, operation='Sum', regard_opponent=True, include_current=False)

#Add difference in corners conceded last five games between the two teams
data['Diff_corners_conceded'] = data['HC_Sum_5_opponent'] - data['AC_Sum_5_opponent']



#Add fouls commited last five games for both home and away team
data = util.add_form_column(data, 'HF', 'AF', n=5, operation='Sum', regard_opponent=False, include_current=False)

#Add difference in corners awarded last five games between the two teams
data['Diff_fouls_commited'] = data['HF_Sum_5'] - data['AF_Sum_5']


#Add fouls suffered last five games for both home and away team
data = util.add_form_column(data, 'HF', 'AF', n=5, operation='Sum', regard_opponent=True, include_current=False)

#Add difference in corners conceded last five games between the two teams
data['Diff_fouls_suffered'] = data['HF_Sum_5_opponent'] - data['AF_Sum_5_opponent']


#Add yellow cards awarded last five games for both home and away team
data = util.add_form_column(data, 'HY', 'AY', n=5, operation='Sum', regard_opponent=False, include_current=False)

#Add difference in yellow cards awarded last five games between the two teams
data['Diff_yellow_cards'] = data['HY_Sum_5'] - data['AY_Sum_5']


#Add red cards awarded last five games for both home and away team
data = util.add_form_column(data, 'HR', 'AR', n=5, operation='Sum', regard_opponent=False, include_current=False)

#Add difference in red cards awarded last five games between the two teams
data['Diff_red_cards'] = data['HR_Sum_5'] - data['AR_Sum_5']

#### Data inspection

Let's look at the data we have created

In [38]:
data.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,Diff_fouls_commited,HF_Sum_5_opponent,AF_Sum_5_opponent,Diff_fouls_suffered,HY_Sum_5,AY_Sum_5,Diff_yellow_cards,HR_Sum_5,AR_Sum_5,Diff_red_cards
0,E0,2005-08-13,Aston Villa,Bolton,2.0,2.0,D,2.0,2.0,D,...,0,0,0,0,0,0,0,0,0,0
1,E0,2005-08-13,Everton,Man United,0.0,2.0,A,0.0,1.0,A,...,0,0,0,0,0,0,0,0,0,0
2,E0,2005-08-13,Fulham,Birmingham,0.0,0.0,D,0.0,0.0,D,...,0,0,0,0,0,0,0,0,0,0
3,E0,2005-08-13,Man City,West Brom,0.0,0.0,D,0.0,0.0,D,...,0,0,0,0,0,0,0,0,0,0
4,E0,2005-08-13,Middlesbrough,Liverpool,0.0,0.0,D,0.0,0.0,D,...,0,0,0,0,0,0,0,0,0,0


Distributions

In [39]:
data.describe()

,Date,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,HF,...,Diff_fouls_commited,HF_Sum_5_opponent,AF_Sum_5_opponent,Diff_fouls_suffered,HY_Sum_5,AY_Sum_5,Diff_yellow_cards,HR_Sum_5,AR_Sum_5,Diff_red_cards
count,38417,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000,...,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000,38417.000000
mean,2014-12-14 13:00:28.768514048,1.447172,1.157222,0.641253,0.507041,12.609340,10.340032,5.260666,4.292527,11.068980,...,0.220814,56.388318,56.651456,-0.263139,7.852227,7.705078,0.147148,0.400994,0.387875,0.013119
min,2005-08-06 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-82.000000,0.000000,0.000000,-79.000000,0.000000,0.000000,-17.000000,0.000000,0.000000,-4.000000
25%,2010-03-06 00:00:00,1.000000,0.000000,0.000000,0.000000,9.000000,7.000000,3.000000,2.000000,8.000000,...,-9.000000,49.000000,50.000000,-9.000000,6.000000,6.000000,-3.000000,0.000000,0.000000,0.000000
50%,2014-12-13 00:00:00,1.000000,1.000000,0.000000,0.000000,12.000000,10.000000,5.000000,4.000000,11.000000,...,0.000000,56.000000,56.000000,0.000000,8.000000,8.000000,0.000000,0.000000,0.000000,0.000000
75%,2019-09-16 00:00:00,2.000000,2.000000,1.000000,1.000000,16.000000,13.000000,7.000000,6.000000,13.000000,...,9.000000,63.000000,64.000000,9.000000,10.000000,10.000000,3.000000,1.000000,1.000000,0.000000
max,2024-05-19 00:00:00,9.000000,9.000000,7.000000,5.000000,43.000000,35.000000,24.000000,20.000000,33.000000,...,87.000000,124.000000,138.000000,77.000000,24.000000,23.000000,18.000000,6.000000,5.000000,6.000000
std,NaN,1.225478,1.096314,0.804243,0.713672,4.834794,4.344031,2.876518,2.507945,3.662132,...,13.439982,10.560245,10.613764,13.398871,3.034913,3.004892,4.025614,0.639155,0.629503,0.888303


Types and columns

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38417 entries, 0 to 38416
Data columns (total 77 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Div                             38417 non-null  object        
 1   Date                            38417 non-null  datetime64[ns]
 2   HomeTeam                        38417 non-null  object        
 3   AwayTeam                        38417 non-null  object        
 4   FTHG                            38417 non-null  float64       
 5   FTAG                            38417 non-null  float64       
 6   FTR                             38417 non-null  object        
 7   HTHG                            38417 non-null  float64       
 8   HTAG                            38417 non-null  float64       
 9   HTR                             38417 non-null  object        
 10  HS                              38417 non-null  float64       
 11  AS

### Column removal

At the moment, there are many columns that are not wanted to create the Match Context. We remove these columns to make the dataset more manageable.

In [41]:
columns_to_remove = [
    "FTR",
    "HTHG",
    "HTAG",
    "HTR",
    "HS",
    "AS",
    "HST",
    "AST",
    "HF",
    "AF",
    "HC",
    "AC",
    "HY",
    "AY",
    "HR",
    "AR",
    "Home ELO",
    "Away ELO",
    "FTHG_Sum_5",
    "FTAG_Sum_5",
    "FTHG_Sum_5_opponent",
    "FTAG_Sum_5_opponent",
    "Home Goal Difference last 5",
    "Away Goal Difference last 5",
    "Home_Points_5",
    "Away_Points_5",
    "Home ELO_Change_5",
    "Away ELO_Change_5",
    "Home ELO_Mean_5_opponent",
    "Away ELO_Mean_5_opponent",
    "HST_Sum_5",
    "AST_Sum_5",
    "HST_Sum_5_opponent",
    "AST_Sum_5_opponent",
    "HS_Sum_5",
    "AS_Sum_5",
    "HS_Sum_5_opponent",
    "AS_Sum_5_opponent",
    "HC_Sum_5",
    "AC_Sum_5",
    "HC_Sum_5_opponent",
    "AC_Sum_5_opponent",
	"HF_Sum_5",
	"AF_Sum_5",
	"HF_Sum_5_opponent",
	"AF_Sum_5_opponent",
    "HY_Sum_5",
    "AY_Sum_5",
    "HR_Sum_5",
    "AR_Sum_5",
]

data.drop(columns=columns_to_remove, inplace=True)

### Row removal

As with the previous notebook, we remove rows from the five first games of the season to not get misrepresenting column-data.

In [42]:
data_without_first_5_games = util.remove_the_first_n_matches_in_a_season_for_each_team(data, n=5)
data_without_first_5_games.info()

number of matches removed:  4414
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34003 entries, 0 to 34002
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Div                             34003 non-null  object        
 1   Date                            34003 non-null  datetime64[ns]
 2   HomeTeam                        34003 non-null  object        
 3   AwayTeam                        34003 non-null  object        
 4   FTHG                            34003 non-null  float64       
 5   FTAG                            34003 non-null  float64       
 6   Season                          34003 non-null  object        
 7   ELO diff                        34003 non-null  float64       
 8   Home_prob_ELO                   34003 non-null  float64       
 9   Draw_prob_ELO                   34003 non-null  float64       
 10  Away_prob_ELO                   34003

Now, we have a dataset with well-defined X and y columns. We store this result in a new file for further use.

In [43]:
data_without_first_5_games.to_csv("../data/DataForModel.csv", index=False)